In [1]:
# importing relevant packages 

import torch 
from torchvision import models
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import glob
import os
import cv2
import matplotlib.pyplot as plt 
import torchvision.transforms as transforms
from torchsummary import summary
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import json
from PIL import Image
import random
import albumentations as A
import glob
from shapely.geometry import Polygon

/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/.Keypoint_detection_notebooks/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
class VGG_model(nn.Module):
    def __init__(self):
        super(VGG_model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
            nn.Conv2d(512,512, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512,50, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d(output_size=(8,8))
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(3200, 300),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Dropout(0.3),
            nn.Linear(300, 32),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
        

In [3]:
model = VGG_model()
model.load_state_dict(torch.load('./Model_VGG_4_5_6_2000_16300_epoch'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
model.eval()

VGG_model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.1, inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): LeakyReLU(negative_slope=0.1, inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.1, inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): LeakyReLU(negative_slope=0.1, inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): LeakyReLU(negative_slope=0.1, inplace=True)
    (14): Conv2d(

In [4]:
def predict_keypoints(img):
    img = cv2.resize(img, (224,224))
    img_tensor = transforms.ToTensor()(img)
    img_mean = img_tensor.mean(dim = (1,2))
    img_std = img_tensor.std(dim = (1,2))
        
    img_normalised = transforms.Normalize(img_mean, img_std)(img_tensor)
    img_normalised = img_normalised.to(device)
    
    key_points = model(img_normalised[None]).flatten().detach().cpu().numpy()
    
    return key_points

In [5]:
def plot_keypoints(img, keypoints):                                                             

    plt.imshow(img)

    x_points = keypoints[0::2]
    y_points = keypoints[1::2]
    plt.scatter(x_points*img.shape[1], y_points*img.shape[0], s = 4, c=(1,0,0))
    plt.show()

In [7]:
dataset_names = ['Grasp_dataset_eyedropper', 'Grasp_dataset_roundbottlelarge', 'Grasp_dataset_roundbottlemedium', 'Grasp_dataset_roundbottlesmall', 'Grasp_dataset_squarebottle', 'Grasp_dataset_testtube']

header = ['weight_reading_1', 'weight_reading_2', 'pressure_reading_1', 'pressure_reading_2', 'force_reading_1', 'force_reading_2', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y', 'label']

current_dir = os. getcwd()

weight_reading_1_arr = []
weight_reading_2_arr = []
pressure_reading_1_arr = []
pressure_reading_2_arr = []
force_reading_1_arr = []
force_reading_2_arr = []

p1_x_arr = []
p1_y_arr = []
p2_x_arr = []
p2_y_arr = []
p3_x_arr = []
p3_y_arr = []
p4_x_arr = []
p4_y_arr = []
p5_x_arr = []
p5_y_arr = []
p6_x_arr = []
p6_y_arr = []
p7_x_arr = []
p7_y_arr = []
p8_x_arr = []
p8_y_arr = []
p9_x_arr = []
p9_y_arr = []
p10_x_arr = []
p10_y_arr = []
p11_x_arr = []
p11_y_arr = []
p12_x_arr = []
p12_y_arr = []
p13_x_arr = []
p13_y_arr = []
p14_x_arr = []
p14_y_arr = []
p15_x_arr = []
p15_y_arr = []
p16_x_arr = []
p16_y_arr = []
label_arr = []


for f in dataset_names:
    
    image_dir = os.path.join(current_dir + '/Grasp_dataset_3', f)
    data_dir = os.path.join(image_dir, f + '.csv')
    df = pd.read_csv(data_dir)
    weight_reading_1_arr = np.concatenate((weight_reading_1_arr, df['weight_reading_1'].values))
    weight_reading_2_arr = np.concatenate((weight_reading_2_arr, df['weight_reading_2'].values))
    pressure_reading_1_arr = np.concatenate((pressure_reading_1_arr, df['pressure_reading_1'].values))
    pressure_reading_2_arr = np.concatenate((pressure_reading_2_arr, df['pressure_reading_2'].values))
    force_reading_1_arr = np.concatenate((force_reading_1_arr, df['force_reading_1'].values))
    force_reading_2_arr = np.concatenate((force_reading_2_arr, df['force_reading_2'].values))
    label_arr = np.concatenate((label_arr, df['label'].values))
    
    j = 0
    
    while j < df.shape[0]:
        image = cv2.imread(os.path.join(image_dir, df.iloc[j]['image_name']))
        keypoints = predict_keypoints(image)
        
#         plot_keypoints(image, keypoints)
        
#         if j == 20:
#             plot_keypoints(image, keypoints)
        

#         plt.imshow(image)

#         plt.scatter(kirigami_x_points*image.shape[1], kirigami_y_points*image.shape[0], s = 4, c=(1,0,0))
#         plt.show()
        
        p1_x_arr.append(keypoints[0])
        p1_y_arr.append(keypoints[1])
        p2_x_arr.append(keypoints[2])
        p2_y_arr.append(keypoints[3])
        p3_x_arr.append(keypoints[4])
        p3_y_arr.append(keypoints[5])
        p4_x_arr.append(keypoints[6])
        p4_y_arr.append(keypoints[7])
        p5_x_arr.append(keypoints[8])
        p5_y_arr.append(keypoints[9])
        p6_x_arr.append(keypoints[10])
        p6_y_arr.append(keypoints[11])
        p7_x_arr.append(keypoints[12])
        p7_y_arr.append(keypoints[13])
        p8_x_arr.append(keypoints[14])
        p8_y_arr.append(keypoints[15])
        p9_x_arr.append(keypoints[16])
        p9_y_arr.append(keypoints[17])
        p10_x_arr.append(keypoints[18])
        p10_y_arr.append(keypoints[19])
        p11_x_arr.append(keypoints[20])
        p11_y_arr.append(keypoints[21])
        p12_x_arr.append(keypoints[22])
        p12_y_arr.append(keypoints[23])
        p13_x_arr.append(keypoints[24])
        p13_y_arr.append(keypoints[25])
        p14_x_arr.append(keypoints[26])
        p14_y_arr.append(keypoints[27])
        p15_x_arr.append(keypoints[28])
        p15_y_arr.append(keypoints[29])
        p16_x_arr.append(keypoints[30])
        p16_y_arr.append(keypoints[31])
        
        j = j + 1
        
#         if j == 2:
#             break
#     break


weight_diff_1_arr = np.array(weight_reading_1_arr) - np.array(force_reading_1_arr)
weight_diff_2_arr = np.array(weight_reading_2_arr) - np.array(force_reading_2_arr)

grasp_dataset = pd.concat([pd.Series(weight_diff_1_arr), pd.Series(weight_diff_2_arr), pd.Series(pressure_reading_1_arr), pd.Series(pressure_reading_2_arr), pd.Series(force_reading_1_arr), pd.Series(force_reading_2_arr), pd.Series(p1_x_arr), pd.Series(p1_y_arr), pd.Series(p2_x_arr), pd.Series(p2_y_arr), pd.Series(p3_x_arr), pd.Series(p3_y_arr), pd.Series(p4_x_arr), pd.Series(p4_y_arr), pd.Series(p5_x_arr), pd.Series(p5_y_arr), pd.Series(p6_x_arr), pd.Series(p6_y_arr), pd.Series(p7_x_arr), pd.Series(p7_y_arr), pd.Series(p8_x_arr), pd.Series(p8_y_arr), pd.Series(p9_x_arr), pd.Series(p9_y_arr), pd.Series(p10_x_arr), pd.Series(p10_y_arr), pd.Series(p11_x_arr), pd.Series(p11_y_arr), pd.Series(p12_x_arr), pd.Series(p12_y_arr), pd.Series(p13_x_arr), pd.Series(p13_y_arr), pd.Series(p14_x_arr), pd.Series(p14_y_arr), pd.Series(p15_x_arr), pd.Series(p15_y_arr), pd.Series(p16_x_arr), pd.Series(p16_y_arr), pd.Series(label_arr)], axis=1, keys=header)


In [8]:
pd.options.display.max_columns = None
print(grasp_dataset.head(2))

   weight_reading_1  weight_reading_2  pressure_reading_1  pressure_reading_2  \
0              60.5             -63.2                20.8                15.7   
1              62.4             -61.3                20.1                14.6   

   force_reading_1  force_reading_2     p1_x      p1_y      p2_x      p2_y  \
0            147.3            147.6  0.15715  0.361531  0.475913  0.157452   
1            178.3            178.5  0.15681  0.359949  0.475905  0.156914   

       p3_x      p3_y      p4_x      p4_y      p5_x      p5_y      p6_x  \
0  0.800085  0.329142  0.804217  0.679848  0.490620  0.877026  0.163155   
1  0.799556  0.330455  0.804078  0.679195  0.490764  0.876640  0.162847   

       p6_y      p7_x      p7_y      p8_x      p8_y      p9_x      p9_y  \
0  0.707143  0.459447  0.242009  0.457443  0.386458  0.454404  0.517327   
1  0.707644  0.459070  0.241901  0.456905  0.386393  0.453880  0.517142   

      p10_x     p10_y     p11_x     p11_y     p12_x     p12_y     p13

In [9]:
grasp_dataset

,weight_reading_1,weight_reading_2,pressure_reading_1,pressure_reading_2,force_reading_1,force_reading_2,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,p5_x,p5_y,p6_x,p6_y,p7_x,p7_y,p8_x,p8_y,p9_x,p9_y,p10_x,p10_y,p11_x,p11_y,p12_x,p12_y,p13_x,p13_y,p14_x,p14_y,p15_x,p15_y,p16_x,p16_y,label
0,60.5,-63.2,20.8,15.7,147.3,147.6,0.157150,0.361531,0.475913,0.157452,0.800085,0.329142,0.804217,0.679848,0.490620,0.877026,0.163155,0.707143,0.459447,0.242009,0.457443,0.386458,0.454404,0.517327,0.462041,0.646254,0.472893,0.783225,0.506434,0.799064,0.508312,0.657355,0.511533,0.528987,0.503506,0.399163,0.493839,0.257683,eyedropper
1,62.4,-61.3,20.1,14.6,178.3,178.5,0.156810,0.359949,0.475905,0.156914,0.799556,0.330455,0.804078,0.679195,0.490764,0.876640,0.162847,0.707644,0.459070,0.241901,0.456905,0.386393,0.453880,0.517142,0.461641,0.645948,0.472723,0.782864,0.506353,0.799067,0.507989,0.657509,0.511101,0.529103,0.503097,0.399187,0.493555,0.257713,eyedropper
2,62.6,-61.7,22.0,17.8,150.0,150.3,0.158945,0.362909,0.476207,0.161429,0.800451,0.327022,0.805081,0.682077,0.490964,0.873621,0.164714,0.704094,0.460723,0.242344,0.459925,0.385897,0.457319,0.516959,0.464650,0.645991,0.474418,0.782387,0.507233,0.798474,0.508771,0.657301,0.511810,0.529263,0.503753,0.399672,0.494247,0.258627,eyedropper
3,63.9,-60.6,19.6,14.7,191.7,188.7,0.156410,0.360602,0.475665,0.156453,0.799973,0.329672,0.804234,0.679375,0.490498,0.878210,0.163064,0.707099,0.459049,0.241851,0.456961,0.386437,0.453939,0.517308,0.461651,0.646188,0.472690,0.783224,0.506257,0.799444,0.508115,0.657637,0.511266,0.529307,0.503184,0.399422,0.493468,0.257765,eyedropper
4,62.6,-61.6,19.6,14.5,194.5,191.8,0.155827,0.360163,0.475439,0.157059,0.799797,0.330252,0.803658,0.678968,0.490295,0.877633,0.162732,0.708221,0.458608,0.242005,0.456382,0.386636,0.453315,0.517467,0.461072,0.646311,0.472231,0.783322,0.505840,0.799491,0.507553,0.657769,0.510688,0.529333,0.502676,0.399366,0.493116,0.257786,eyedropper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,66.5,-59.5,16.5,28.7,213.9,211.7,0.159419,0.366107,0.475965,0.152067,0.799201,0.325419,0.802661,0.677922,0.489941,0.885915,0.164303,0.706277,0.460682,0.242528,0.458640,0.387592,0.455522,0.519432,0.462739,0.648777,0.473200,0.786145,0.506568,0.801165,0.509746,0.657821,0.513446,0.529890,0.505069,0.400166,0.494440,0.257211,testtube
596,64.9,-62.6,19.1,14.1,186.6,190.2,0.158052,0.366277,0.476729,0.148116,0.799829,0.325745,0.800959,0.676130,0.488899,0.885519,0.160965,0.713363,0.459942,0.240797,0.456236,0.387459,0.452310,0.518904,0.459686,0.647702,0.471229,0.785295,0.504889,0.800859,0.508486,0.657628,0.512679,0.529188,0.504812,0.399063,0.494470,0.255959,testtube
597,64.2,-62.2,20.6,14.7,197.2,197.3,0.156129,0.359739,0.475387,0.147543,0.798490,0.330694,0.802470,0.678588,0.490446,0.883455,0.162534,0.706832,0.458452,0.240408,0.455490,0.386435,0.452347,0.517730,0.460358,0.646582,0.472301,0.784034,0.506408,0.800376,0.509119,0.657723,0.512579,0.529643,0.504121,0.399625,0.493425,0.256367,testtube
598,64.1,-62.0,19.7,14.6,215.9,216.6,0.156209,0.360668,0.475462,0.147921,0.798693,0.330461,0.802300,0.678278,0.490461,0.883958,0.162204,0.707331,0.458545,0.240665,0.455469,0.386837,0.452230,0.518077,0.460252,0.646862,0.472288,0.784392,0.506281,0.800577,0.509010,0.657896,0.512488,0.529771,0.504071,0.399744,0.493424,0.256572,testtube


In [10]:
grasp_data = grasp_dataset.drop(columns=['label'], axis=1)
# grasp_data = grasp_dataset.drop(columns=['label', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y'], axis=1)
grasp_data['weight_reading_1'] = grasp_data['weight_reading_1'] / 1000
grasp_data['weight_reading_2'] = grasp_data['weight_reading_2'] / 1000
grasp_data['pressure_reading_1'] = grasp_data['pressure_reading_1'] / 100
grasp_data['pressure_reading_2'] = grasp_data['pressure_reading_2'] / 100
grasp_data['force_reading_1'] = grasp_data['force_reading_1'] / 1000
grasp_data['force_reading_2'] = grasp_data['force_reading_2'] / 1000
grasp_label = grasp_dataset['label']

print(grasp_data.head(2))
print(grasp_label.head(2))

   weight_reading_1  weight_reading_2  pressure_reading_1  pressure_reading_2  \
0            0.0605           -0.0632               0.208               0.157   
1            0.0624           -0.0613               0.201               0.146   

   force_reading_1  force_reading_2     p1_x      p1_y      p2_x      p2_y  \
0           0.1473           0.1476  0.15715  0.361531  0.475913  0.157452   
1           0.1783           0.1785  0.15681  0.359949  0.475905  0.156914   

       p3_x      p3_y      p4_x      p4_y      p5_x      p5_y      p6_x  \
0  0.800085  0.329142  0.804217  0.679848  0.490620  0.877026  0.163155   
1  0.799556  0.330455  0.804078  0.679195  0.490764  0.876640  0.162847   

       p6_y      p7_x      p7_y      p8_x      p8_y      p9_x      p9_y  \
0  0.707143  0.459447  0.242009  0.457443  0.386458  0.454404  0.517327   
1  0.707644  0.459070  0.241901  0.456905  0.386393  0.453880  0.517142   

      p10_x     p10_y     p11_x     p11_y     p12_x     p12_y     p13

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

data_train, data_test, label_train, label_test = train_test_split(grasp_data, grasp_label, test_size=0.2, random_state=np.random.randint(100))

# print(X_train.head(3))

# clf = KNeighborsClassifier(n_neighbors=7)

# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = knn.score(X_test, y_test)
# print('Accuracy:', accuracy)
# clf = RandomForestClassifier(n_estimators=1000, max_depth=50, random_state=np.random.randint(100))
clf = RandomForestClassifier(n_estimators=1000, max_depth=200, random_state=np.random.randint(100), n_jobs = -1)
# clf = HistGradientBoostingClassifier(max_iter=200, random_state=np.random.randint(100))
# clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=1, random_state=0)
# clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')

num_folds = 10
cv_method = KFold(n_splits=num_folds, shuffle=True, random_state=np.random.randint(100))
cv_results = cross_val_score(clf, grasp_data, grasp_label, cv=cv_method, scoring='accuracy')

print('Cross-validation results:', cv_results)
print('Average accuracy:', cv_results.mean())

# clf.fit(data_train, label_train)
# label_pred = clf.predict(data_test)
# cm = confusion_matrix(label_test, label_pred)

# short_dataset_names = []'eyedropper', 'roundbottlelarge', 'roundbottlemedium', 'roundbottlesmall', 'squarebottle', 'testtube']

# cm_df = pd.DataFrame(cm, index=short_dataset_names, columns=short_dataset_names)

# print(cm_df)
# print(cm)

# accuracy = clf.score(data_test, label_test)
# print("Accuracy:", accuracy)

Cross-validation results: [0.98333333 1.         1.         1.         1.         1.
 1.         1.         1.         1.        ]
Average accuracy: 0.9983333333333334
